## Asynchronous Execution

You can define a task to be executed asynchronously. This means that the crew will not wait for it to be completed to continue with the next task. This is useful for tasks that take a long time to be completed, or that are not crucial for the next tasks to be performed.
You can then use the context attribute to define in a future task that it should wait for the output of the asynchronous task to be completed.

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from crewai import Agent,LLM,Crew,Task
from crewai_tools import TavilySearchTool
from crewai.tools import tool

In [3]:
llm = LLM(model="groq/llama-3.3-70b-versatile", temperature=0.2)

In [4]:
search_tool = TavilySearchTool(max_results=2,topic="finance")

In [8]:
market_analysis_agent = Agent(
    role="Senior Stock Market Analyst",
    goal="Perform deep financial and market analysis for the stock symbol {symbol}.",
    backstory="""
    You are a senior equity research analyst with 15+ years of experience.
    You specialize in analyzing company financials, profitability, valuation metrics,
    revenue growth, earnings reports, and overall market trends.
    You provide structured, professional financial insights.
    """,
    tools=[search_tool],
    llm=llm,
    max_tokens=500,
    verbose=True
)


In [9]:
future_trend_agent = Agent(
    role="Investment Forecast Strategist",
    goal="Predict short-term future outlook and hype potential of stock {symbol}.",
    backstory="""
    You are a financial forecasting expert skilled in analyzing
    market sentiment, stock momentum, macroeconomic signals,
    institutional activity, and growth catalysts.
    You provide realistic, probability-based forecasts.
    """,
    tools=[search_tool],
    llm=llm,
    max_tokens=500,
    verbose=True
)


In [10]:
risk_analysis_agent = Agent(
    role="Financial Risk Assessment Specialist",
    goal="Identify risks, weaknesses, and potential threats for stock {symbol}.",
    backstory="""
    You are a risk auditor who identifies volatility risks,
    regulatory threats, competition challenges, lawsuits,
    declining revenues, and economic pressure.
    You focus on downside protection analysis.
    """,
    tools=[search_tool],
    llm=llm,
    max_tokens=500,
    verbose=True
)


In [11]:
report_aggregator_agent = Agent(
    role="Chief Investment Report Strategist",
    goal="Aggregate all stock analysis and generate final professional report for {symbol}.",
    backstory="""
    You are a senior financial strategist responsible for preparing
    executive-level stock evaluation reports.
    You combine multiple expert analyses into one clear,
    structured and actionable investment report.
    """,
    tools=[search_tool],
    llm=llm,
    max_tokens=500,
    verbose=True
)


In [12]:
market_analysis_task = Task(
    description="""
    Perform detailed financial and market analysis for stock symbol {symbol}.

    Include:
    - Current market trend
    - Revenue and profit performance
    - Financial metrics (P/E, EPS, Market Cap if available)
    - Recent quarterly performance
    - Overall financial health
    """,
    expected_output="Structured financial analysis report.",
    agent=market_analysis_agent,
    async_execution=True # first 
)


In [13]:
future_trend_task = Task(
    description="""
    Analyze the future outlook of stock {symbol}.

    Include:
    - Short-term prediction (next 2 months)
    - Growth potential
    - Market hype level (High/Moderate/Low)
    - Investment attractiveness
    """,
    expected_output="Future investment outlook report.",
    agent=future_trend_agent,
    async_execution=True 
)


In [14]:
risk_analysis_task = Task(
    description="""
    Perform risk and downside analysis for stock {symbol}.

    Include:
    - Recent losses
    - Industry competition threats
    - Regulatory risks
    - Market volatility
    - Any current issues
    """,
    expected_output = "Detailed risk analysis report.",
    agent = risk_analysis_agent,
    async_execution = True
)


In [15]:
final_report_task = Task(
    description="""
    Using all previous analyses, generate a comprehensive investment report for {symbol}.

    The report must include:

    1. Executive Summary
    2. Market Performance Overview
    3. Future Outlook (Next 2 Months)
    4. Risk Assessment
    5. Final Recommendation (Buy / Hold / Sell with justification)

    Make it professional and investor-ready.
    """,
    expected_output="Complete stock evaluation report.",
    agent=report_aggregator_agent,
    context=[
        market_analysis_task,
        future_trend_task,
        risk_analysis_task
    ]
)


In [16]:
from crewai import Process

In [17]:
crew = Crew(
    agents=[
        market_analysis_agent,
        future_trend_agent,
        risk_analysis_agent,
        report_aggregator_agent
    ],
    tasks=[
        market_analysis_task,
        future_trend_task,
        risk_analysis_task,
        final_report_task
    ],
    verbose=True
)

In [18]:
result = crew.kickoff(inputs={"symbol": "AAPL"})   # inputs={"symbol": "TSLA"}
print(result)

╭─────────────────────────────────────────── 🚀 Crew Execution Started ───────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name:                                                                                                          │
│  crew                                                                                                           │
│  ID:                                                                                                            │
│  260a4c27-9309-4964-9b33-72cc18d43c73                                                                           │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── 📋 Task Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Started                                                                                                   │
│  Name:                                                                                                          │
│      Perform detailed financial and market analysis for stock symbol AAPL.                                      │
│                                                                                                                 │
│      Include:                                                                                                   │
│      - Current market trend                                                                                     │
│      - Revenue and profit performance                                                                           │
│      - Financial metrics (P/E, EPS, Market Cap if available)                                                    │
│      - Recent quarterly performance                                                                             │
│      - Overall financial health                                                                                 │
│                                                                                                                 │
│  ID: b564e8da-2593-48a2-91a7-7a2aa48631d0                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── 📋 Task Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Started                                                                                                   │
│  Name:                                                                                                          │
│      Analyze the future outlook of stock AAPL.                                                                  │
│                                                                                                                 │
│      Include:                                                                                                   │
│      - Short-term prediction (next 2 months)                                                                    │
│      - Growth potential                                                                                         │
│      - Market hype level (High/Moderate/Low)                                                                    │
│      - Investment attractiveness                                                                                │
│                                                                                                                 │
│  ID: 02d540f1-1805-4a81-9a52-c81b21ab80f8                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── 📋 Task Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Started                                                                                                   │
│  Name:                                                                                                          │
│      Perform risk and downside analysis for stock AAPL.                                                         │
│                                                                                                                 │
│      Include:                                                                                                   │
│      - Recent losses                                                                                            │
│      - Industry competition threats                                                                             │
│      - Regulatory risks                                                                                         │
│      - Market volatility                                                                                        │
│      - Any current issues                                                                                       │
│                                                                                                                 │
│  ID: 0dfd27f7-84a8-4c01-a920-1eda5d6ee097                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Senior Stock Market Analyst                                                                             │
│                                                                                                                 │
│  Task:                                                                                                          │
│      Perform detailed financial and market analysis for stock symbol AAPL.                                      │
│                                                                                                                 │
│      Include:                                                                                                   │
│      - Current market trend                                                                                     │
│      - Revenue and profit performance                                                                           │
│      - Financial metrics (P/E, EPS, Market Cap if available)                                                    │
│      - Recent quarterly performance                                                                             │
│      - Overall financial health                                                                                 │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

[CrewAIEventsBus] Warning: Event pairing mismatch. 'agent_execution_error' closed 'llm_call_started' (expected 
'agent_execution_started')

╭───────────────────────────────────────────────── ❌ LLM Error ──────────────────────────────────────────────────╮
│                                                                                                                 │
│  LLM Call Failed                                                                                                │
│  Error: litellm.RateLimitError: RateLimitError: GroqException - {"error":{"message":"Rate limit reached for     │
│  model `llama-3.3-70b-versatile` in organization `org_01jc8r55vqeq6v4yza1hzy47vz` service tier `on_demand` on   │
│  tokens per minute (TPM): Limit 12000, Used 10796, Requested 2708. Please try again in 7.52s. Need more         │
│  tokens? Upgrade to Dev Tier today at                                                                           │
│  https://console.groq.com/settings/billing","type":"tokens","code":"rate_limit_exceeded"}}                      │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

[CrewAIEventsBus] Warning: Event pairing mismatch. 'task_failed' closed 'llm_call_started' (expected 
'task_started')

╭──────────────────────────────────────────────── 📋 Task Failure ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Failed                                                                                                    │
│  Name:                                                                                                          │
│                                                                                                                 │
│      Perform risk and downside analysis for stock AAPL.                                                         │
│                                                                                                                 │
│      Include:                                                                                                   │
│      - Recent losses                                                                                            │
│      - Industry competition threats                                                                             │
│      - Regulatory risks                                                                                         │
│      - Market volatility                                                                                        │
│      - Any current issues                                                                                       │
│                                                                                                                 │
│  Agent:                                                                                                         │
│  Financial Risk Assessment Specialist                                                                           │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

RateLimitError: litellm.RateLimitError: RateLimitError: GroqException - {"error":{"message":"Rate limit reached for model `llama-3.3-70b-versatile` in organization `org_01jc8r55vqeq6v4yza1hzy47vz` service tier `on_demand` on tokens per minute (TPM): Limit 12000, Used 10779, Requested 2840. Please try again in 8.094999999s. Need more tokens? Upgrade to Dev Tier today at https://console.groq.com/settings/billing","type":"tokens","code":"rate_limit_exceeded"}}
